## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
# Drop the 'EIN' and 'NAME' columns
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

In [3]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()
unique_values

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type = application_df['APPLICATION_TYPE'].value_counts()
application_type

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 500

application_types_to_replace = application_type[application_type < cutoff].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_1 = classification_counts[classification_counts > 1]
classification_counts_1

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 1000

classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_numeric = pd.get_dummies(application_df)

In [10]:
# Split our preprocessed data into our features and target arrays
X = application_df_numeric.drop(columns='IS_SUCCESSFUL')
y = application_df_numeric['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential([

# First hidden layer
tf.keras.layers.Dense(units=10, activation='relu', input_dim=X_train.shape[1]),

# Second hidden layer
tf.keras.layers.Dense(units=5, activation='relu'),

# Output layer
tf.keras.layers.Dense(units=1, activation='sigmoid')
])

# Check the structure of the model
nn.summary()

/Users/ryannuttall/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 10)             │           440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │            55 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 501 (1.96 KB)

 Trainable params: 501 (1.96 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Compile the model
nn.compile(optimizer='adam',
           loss='binary_crossentropy',
           metrics=['accuracy'])

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 230us/step - accuracy: 0.6196 - loss: 0.6849
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 224us/step - accuracy: 0.7245 - loss: 0.5663
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 224us/step - accuracy: 0.7288 - loss: 0.5597
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 223us/step - accuracy: 0.7331 - loss: 0.5535
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 226us/step - accuracy: 0.7304 - loss: 0.5537
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 222us/step - accuracy: 0.7287 - loss: 0.5543
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 223us/step - accuracy: 0.7326 - loss: 0.5506
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 223us/step - accuracy: 0.7346 - loss: 0.5482
Epoch 9/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 222us/step - accuracy: 0.7312 - loss: 0.5494
Epoch 10/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 228us/step - accuracy: 0.7297 - loss: 0.5494
Epoch 11/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 221us/step - accuracy: 0.7327 - loss: 0.5505
Epoch 12/100
858/85

858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 233us/step - accuracy: 0.7350 - loss: 0.5425
Epoch 68/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 223us/step - accuracy: 0.7315 - loss: 0.5428
Epoch 69/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 223us/step - accuracy: 0.7344 - loss: 0.5405
Epoch 70/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 223us/step - accuracy: 0.7332 - loss: 0.5425
Epoch 71/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 223us/step - accuracy: 0.7301 - loss: 0.5462
Epoch 72/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 222us/step - accuracy: 0.7318 - loss: 0.5421
Epoch 73/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 222us/step - accuracy: 0.7352 - loss: 0.5392
Epoch 74/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 222us/step - accuracy: 0.7350 - loss: 0.5404
Epoch 75/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 223us/step - accuracy: 0.7380 - loss: 0.5377
Epoch 76/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 223us/step - accuracy: 0.7374 - loss: 0.5420
Epoch 77/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 223us/step - accuracy: 0.7332 - loss: 0.5424
Epoch 78/100
858/858 ━━

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 396us/step - accuracy: 0.7254 - loss: 0.5622
Loss: 0.5622454285621643, Accuracy: 0.7253644466400146


In [19]:
# Export our model to HDF5 file
model_HDF5 = 'HDF5.h5'

nn.save(model_HDF5)